REAL/FAKE JOB POSTING PREDICTION

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# WordCloud
from wordcloud import WordCloud

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

True

1. LOAD DATASET

In [ ]:
data = pd.read_csv("fake_job_postings.csv")
print(f"Dataset loaded: {data.shape[0]} rows, {data.shape[1]} columns")

Dataset loaded: 17880 rows, 18 columns


2. DATA EXPLORATION

In [ ]:
print("\n[2] Data Exploration")
print(f"\nFraudulent distribution:\n{data['fraudulent'].value_counts()}")
print(f"\nMissing values:\n{data.isnull().sum().sum()} total missing values")


[2] Data Exploration

Fraudulent distribution:
fraudulent
0    17014
1      866
Name: count, dtype: int64

Missing values:
70106 total missing values


3. DATA PREPROCESSING

In [ ]:
data['text'] = (data['title'].fillna('') + ' ' +
                data['location'].fillna('') + ' ' +
                data['department'].fillna('') + ' ' +
                data['company_profile'].fillna('') + ' ' +
                data['description'].fillna('') + ' ' +
                data['requirements'].fillna('') + ' ' +
                data['benefits'].fillna('') + ' ' +
                data['employment_type'].fillna('') + ' ' +
                data['required_experience'].fillna('') + ' ' +
                data['required_education'].fillna('') + ' ' +
                data['industry'].fillna('') + ' ' +
                data['function'].fillna(''))

data.fillna(' ', inplace=True)

In [ ]:
# Removing punctuation
def remove_punctuation(input_string):
    translator = str.maketrans('', '', string.punctuation)
    cleaned_string = re.sub(r'\s+', ' ', input_string)
    return cleaned_string.translate(translator)

data['text'] = data['text'].apply(remove_punctuation)

# Removing stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

data['text'] = data['text'].apply(remove_stopwords)

In [ ]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_sentence(sentence):
    words = word_tokenize(sentence)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

data['text'] = data['text'].apply(lemmatize_sentence)

# Cleaning text
def clean_text(text):
    return text.strip().lower()

data['text'] = data['text'].apply(clean_text)

4. DATA VISUALIZATION

In [ ]:
# Fraud distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='fraudulent', data=data)
plt.title('Real vs Fraudulent Job Postings', fontsize=16, fontweight='bold')
plt.xlabel('Job Type (0=Real, 1=Fraudulent)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks([0, 1], ['Real', 'Fraudulent'])
plt.tight_layout()
plt.savefig('fraud_distribution.png', dpi=300, bbox_inches='tight')
plt.close()


In [ ]:
def split(location):
    if pd.isna(location) or location == ' ':
        return 'Unknown'
    l = location.split(',')
    return l[0] if l[0] else 'Unknown'

data['country'] = data['location'].apply(split)
country = dict(data['country'].value_counts()[:11])
if ' ' in country:
    del country[' ']

plt.figure(figsize=(10, 6))
plt.title('Country-wise Job Posting', size=16, fontweight='bold')
plt.bar(country.keys(), country.values(), color='steelblue')
plt.ylabel('No. of jobs', size=12)
plt.xlabel('Countries', size=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('country_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
# Experience distribution
experience = dict(data['required_experience'].value_counts())
if ' ' in experience:
    del experience[' ']

plt.figure(figsize=(10, 6))
plt.bar(experience.keys(), experience.values(), color='coral')
plt.title('No. of Jobs with Experience', fontsize=16, fontweight='bold')
plt.xlabel('Experience', size=12)
plt.ylabel('No. of jobs', size=12)
plt.xticks(rotation=35, ha='right')
plt.tight_layout()
plt.savefig('experience_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
# Word clouds
fraudjobs_text = data[data['fraudulent'] == 1]['text'].values
actualjobs_text = data[data['fraudulent'] == 0]['text'].values

plt.figure(figsize=(16, 10))
wc = WordCloud(width=1600, height=800, background_color='white',
               max_words=200, colormap='Reds').generate(str(" ".join(fraudjobs_text)))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title('Fraudulent Jobs Word Cloud', fontsize=20, fontweight='bold')
plt.tight_layout()
plt.savefig('wordcloud_fraudulent.png', dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(16, 10))
wc = WordCloud(width=1600, height=800, background_color='white',
               max_words=200, colormap='Blues').generate(str(" ".join(actualjobs_text)))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title('Real Jobs Word Cloud', fontsize=20, fontweight='bold')
plt.tight_layout()
plt.savefig('wordcloud_real.png', dpi=300, bbox_inches='tight')
plt.close()

 5. FEATURE EXTRACTION

In [ ]:
X = data['text']
y = data['fraudulent']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42, stratify=y)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.8, ngram_range=(1, 2))
cv_train = tfidf.fit_transform(X_train)
cv_test = tfidf.transform(X_test)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Feature matrix shape: {cv_train.shape}")

Training set: 12516 samples
Test set: 5364 samples
Feature matrix shape: (12516, 5000)


 6. MODEL TRAINING

In [ ]:
results = {}

# Model 1: Multinomial Naive Bayes
print("\n" + "="*80)
print("MODEL 1: MULTINOMIAL NAIVE BAYES")
print("="*80)
mnb = MultinomialNB()
mnb.fit(cv_train, y_train)
y_pred_mnb = mnb.predict(cv_test)
acc_mnb = accuracy_score(y_test, y_pred_mnb)

print(f"\nClassification Accuracy: {acc_mnb:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_mnb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_mnb))

results['MultinomialNB'] = acc_mnb


MODEL 1: MULTINOMIAL NAIVE BAYES

Classification Accuracy: 0.9698

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5104
           1       0.97      0.39      0.55       260

    accuracy                           0.97      5364
   macro avg       0.97      0.69      0.77      5364
weighted avg       0.97      0.97      0.96      5364

Confusion Matrix:
[[5101    3]
 [ 159  101]]


In [ ]:
# Model 2: Support Vector Machine
print("\n" + "="*80)
print("MODEL 2: SUPPORT VECTOR MACHINE")
print("="*80)
svc = SVC(kernel='linear', random_state=42)
svc.fit(cv_train, y_train)
y_pred_svc = svc.predict(cv_test)
acc_svc = accuracy_score(y_test, y_pred_svc)

print(f"\nClassification Accuracy: {acc_svc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svc))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svc))

results['SVM'] = acc_svc


MODEL 2: SUPPORT VECTOR MACHINE

Classification Accuracy: 0.9838

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5104
           1       0.98      0.68      0.80       260

    accuracy                           0.98      5364
   macro avg       0.98      0.84      0.90      5364
weighted avg       0.98      0.98      0.98      5364

Confusion Matrix:
[[5100    4]
 [  83  177]]


In [ ]:
# Model 3: Logistic Regression
print("\n" + "="*80)
print("MODEL 3: LOGISTIC REGRESSION")
print("="*80)
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(cv_train, y_train)
y_pred_lr = logreg.predict(cv_test)
acc_lr = accuracy_score(y_test, y_pred_lr)

print(f"\nClassification Accuracy: {acc_lr:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))

results['Logistic Regression'] = acc_lr


MODEL 3: LOGISTIC REGRESSION

Classification Accuracy: 0.9733

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5104
           1       0.99      0.45      0.62       260

    accuracy                           0.97      5364
   macro avg       0.98      0.73      0.80      5364
weighted avg       0.97      0.97      0.97      5364

Confusion Matrix:
[[5103    1]
 [ 142  118]]


7. MODEL COMPARISON

In [ ]:
print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)

comparison_df = pd.DataFrame(list(results.items()),
                            columns=['Model', 'Accuracy']).sort_values('Accuracy', ascending=False)
print("\n", comparison_df.to_string(index=False))

# Visualization
plt.figure(figsize=(10, 6))
plt.bar(comparison_df['Model'], comparison_df['Accuracy'],
        color=['#2ecc71', '#3498db', '#e74c3c'])
plt.title('Model Accuracy Comparison', fontsize=16, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Model', fontsize=12)
plt.ylim([0.95, 1.0])
for i, (model, acc) in enumerate(zip(comparison_df['Model'], comparison_df['Accuracy'])):
    plt.text(i, acc + 0.002, f'{acc:.4f}', ha='center', fontsize=11, fontweight='bold')
plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.close()


MODEL COMPARISON

               Model  Accuracy
                SVM  0.983781
Logistic Regression  0.973341
      MultinomialNB  0.969799


In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

cm_mnb = confusion_matrix(y_test, y_pred_mnb)
sns.heatmap(cm_mnb, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Real', 'Fraudulent'], yticklabels=['Real', 'Fraudulent'])
axes[0].set_title(f'Multinomial NB\nAccuracy: {acc_mnb:.4f}', fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

cm_svc = confusion_matrix(y_test, y_pred_svc)
sns.heatmap(cm_svc, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['Real', 'Fraudulent'], yticklabels=['Real', 'Fraudulent'])
axes[1].set_title(f'SVM\nAccuracy: {acc_svc:.4f}', fontweight='bold')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[2],
            xticklabels=['Real', 'Fraudulent'], yticklabels=['Real', 'Fraudulent'])
axes[2].set_title(f'Logistic Regression\nAccuracy: {acc_lr:.4f}', fontweight='bold')
axes[2].set_ylabel('True Label')
axes[2].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.close()

8. FINAL SUMMARY

In [ ]:
best_model = comparison_df.iloc[0]['Model']
best_accuracy = comparison_df.iloc[0]['Accuracy']

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)
print(f"\n Best Model: {best_model}")
print(f"   Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"\nTotal job postings: {len(data)}")
print(f"Fraudulent: {len(data[data['fraudulent']==1])} ({len(data[data['fraudulent']==1])/len(data)*100:.2f}%)")
print(f"Real: {len(data[data['fraudulent']==0])} ({len(data[data['fraudulent']==0])/len(data)*100:.2f}%)")

print("\n📊 Generated Files:")
print("  1. fraud_distribution.png")
print("  2. country_distribution.png")
print("  3. experience_distribution.png")
print("  4. wordcloud_fraudulent.png")
print("  5. wordcloud_real.png")
print("  6. model_comparison.png")
print("  7. confusion_matrices.png")


FINAL RESULTS

 Best Model: SVM
   Accuracy: 0.9838 (98.38%)

Total job postings: 17880
Fraudulent: 866 (4.84%)
Real: 17014 (95.16%)

📊 Generated Files:
  1. fraud_distribution.png
  2. country_distribution.png
  3. experience_distribution.png
  4. wordcloud_fraudulent.png
  5. wordcloud_real.png
  6. model_comparison.png
  7. confusion_matrices.png
